In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# load data for hard failure
#"log_red2_red_1.pkl", "log_red2.pkl"
with open("log_red2_red_1.pkl", "rb") as f:
    data = pickle.load(f)


In [3]:
print(len(data[0]), len(data[1]), len(data[2]), len(data[3]))
primary_board_ind = 1
redundant_board_ind = 2

56 25 58 57


In [4]:
data_format = ('board_id', 'pkt_no', 'Tx_time', 'reTx_time', 'RSSI', 'payload')

In [5]:
### Number of Packets received by primary board and redundant board
print('Primary Board:', len(data[primary_board_ind]))
print('Redundant Board:', len(data[redundant_board_ind]))


Primary Board: 25
Redundant Board: 58


In [6]:
data[1]

[(94420780, 1, 74, 74, -98),
 (94420780, 2, 110, 110, -98),
 (94420780, 3, 144, 144, -99),
 (94420780, 4, 184, 184, -102),
 (94420780, 6, 232, 232, -101),
 (94420780, 9, 313, 313, -95),
 (94420780, 11, 369, 369, -102),
 (94420780, 15, 492, 492, -100),
 (94420780, 17, 561, 561, -98),
 (94420780, 18, 596, 596, -96),
 (94420780, 1, 61, 61, -98),
 (94420780, 2, 98, 98, -97),
 (94420780, 3, 134, 134, -37),
 (94420780, 4, 171, 171, -98),
 (94420780, 7, 286, 286, -102),
 (94420780, 9, 338, 338, -99),
 (94420780, 14, 472, 472, -100),
 (94420780, 15, 506, 506, -94),
 (94420780, 17, 568, 568, -96),
 (94420780, 19, 623, 623, -98),
 (94420780, 20, 651, 651, -100),
 (94420780, 22, 733, 733, -92),
 (94420780, 23, 753, 753, -95),
 (94420780, 24, 785, 785, -98),
 (94420780, 25, 826, 826, -31)]

In [7]:
### check if all packets belong to same board
### PB
pb_id = 0
rb_id = 0
for d_i, d in enumerate(data[primary_board_ind]):
    if d_i == 0:
        pb_id = d[0]
        print('PB Id:',pb_id) 
    assert(data[primary_board_ind][0][0]==d[0])
### RB
for d_i, d in enumerate(data[redundant_board_ind]):
    if d_i == 0:
        rb_id = d[0]
        print('RB Id:', rb_id)
    assert(data[redundant_board_ind][0][0]==d[0])

PB Id: 94420780
RB Id: 2750291925


#### Checking the confidence interval (40s)


In [8]:
data_all = data[primary_board_ind] + data[redundant_board_ind]
print(len(data_all))

83


In [9]:
for d in data_all:
    if d[3]-d[2]>0:
        #print(d[1],d[4:6], d[5]-d[4])
        print('packets outside conf. interval', d)

#### PRR of the sensor node as whole (PB+RB)


In [10]:
### count lost packets and calculate PRR (APPROach 1)
hb_pkts = []
data_pkts_rb = []
board_index = [primary_board_ind, redundant_board_ind]
pkts_lost_pb = 0
pkts_lost_rb = 0
pb_reset_count = 0
rb_reset_count = 0
pkts_tx = []
pkts_tx_rb = []
hb_indicator = 0
# iterate over boards
for i in board_index:
    each_board = data[i]
    if i == primary_board_ind:
        previous_id = 0
        for d_i, d in enumerate(each_board):
            print(d[1])
            if d[1] - previous_id > 1:
                print(d[1], previous_id)
                pkts_lost_pb += (d[1] - previous_id - 1)
            if d[1] - previous_id < 0:
                pb_reset_count += 1
                pkts_tx += [previous_id]
            previous_id = d[1]
            if d_i == len(each_board) - 1:
                pkts_tx += [d[1]]
    if i == redundant_board_ind:
        previous_id = 0
        data_pkts_rb = []
        for d in each_board:
            #print(d[0][0])
            #indicate if the packet is heartbeat pkt (2 conditions depending upon pkt format)
            if len(d) == 5:
                if d[1] == 0:
                    hb_indicator = 1
                else:
                    hb_indicator = 0
            else:
                if d[5][0] == -1:
                    hb_indicator = 1
                else:
                    hb_indicator = 0
            #seperate data and hb pkts        
            if hb_indicator == 1:
                hb_pkts += [d]
            else:
                print(d[1])
                # check if pkts lost
                if d[1] - previous_id > 1:
                    # for redundant board,, only check after board has failed
                    if d[1] > pkts_tx[0] :
                        print(d[1], previous_id)
                        pkts_lost_rb += (d[1] - previous_id - 1)
                # check if board has restarted
                if d[1] - previous_id < 0:
                    rb_reset_count += 1
                    pkts_tx_rb += [previous_id]
                    break   # dont want o count pkt loss after pb is back online
                data_pkts_rb += [d]
                previous_id = d[1]
                # store the last packet sent by the board
                if d_i == len(each_board) - 1:
                    pkts_tx_rb += [d[1]]

### PRR with Redundancy (PB+RB)
total_pkts = pkts_tx_rb[-1] + pkts_tx[-1]            ### last packet of redundant board and 2nd session of PB
total_pkts_lost = pkts_lost_pb+pkts_lost_rb          ### number of packets lost during tx
prr_sn = ((total_pkts-total_pkts_lost)/total_pkts) * 100
print('PRR with Redundancy:', prr_sn)

### PRR without Redundancy (PB)
prr_pb = ((np.sum(pkts_tx)-pkts_lost_pb)/total_pkts) * 100
print('PRR without Redundancy:', prr_pb)
                    

1
2
3
4
6
6 4
9
9 6
11
11 9
15
15 11
17
17 15
18
1
2
3
4
7
7 4
9
9 7
14
14 9
15
17
17 15
19
19 17
20
22
22 20
23
24
25
1
5
6
7
8
9
11
12
18
19
20
21
22
24
24 22
25
27
27 25
28
29
31
31 29
32
33
35
35 33
36
37
39
39 37
40
41
44
44 41
45
46
47
49
49 47
50
51
52
54
54 52
56
56 54
58
58 56
59
5
PRR with Redundancy: 65.47619047619048
PRR without Redundancy: 29.761904761904763


In [11]:
pkts_tx

[18, 25]

In [12]:
pkts_tx_rb

[59]

In [13]:
pb_reset_count

1

In [14]:
### PRR with Redundancy (PB+RB)
packet_num = []
num_pkts_sent = 0 
num_pkts_rx = 0
for d in data_pkts:
    if d[1] not in packet_num:
        packet_num += [d[1]]
num_pkts_sent = max(packet_num)
num_pkts_rx = len(packet_num)
prr_sn = (num_pkts_rx/num_pkts_sent) * 100
print('PRR of the sensor node:', prr_sn)

### PRR without Redundancy (PR)


NameError: name 'data_pkts' is not defined

#### Efficiency of RB 


In [30]:
data[1]

[(94420780,
  0,
  31,
  31,
  -104,
  (585.0999755859375,
   0.0,
   23.304964065551758,
   1009.3259887695312,
   22.600000381469727,
   39.400001525878906,
   22.80000114440918,
   46.20000076293945,
   22.80000114440918,
   47.0,
   22.899999618530273,
   46.5,
   -157.0,
   0,
   0,
   0,
   94420780)),
 (94420780,
  1,
  61,
  61,
  -103,
  (601.4099731445312,
   0.0,
   22.413785934448242,
   1009.2979736328125,
   22.600000381469727,
   39.20000076293945,
   22.899999618530273,
   46.20000076293945,
   22.899999618530273,
   46.70000076293945,
   22.899999618530273,
   46.400001525878906,
   -44.0,
   0,
   0,
   1,
   94420780)),
 (94420780,
  2,
  91,
  91,
  -94,
  (595.6400146484375,
   0.0,
   22.071962356567383,
   1009.27734375,
   22.700000762939453,
   39.0,
   22.899999618530273,
   45.900001525878906,
   22.899999618530273,
   46.400001525878906,
   22.899999618530273,
   46.10000228881836,
   -44.0,
   0,
   0,
   2,
   94420780)),
 (94420780,
  3,
  121,
  121,
  -

In [19]:
pkts_tx

[18, 25]

In [17]:
faulty_detected_pkts = []
lost_detected_pkts = []
        
for d in data_pkts_rb:
    if d[1] < pkts_tx[0]:   ### 20: last pkt by PB before failure, 47: last pkt by RB bfore resetting pkt count
        lost_detected_pkts += [d]
    else:
        faulty_detected_pkts += [d]

detection_rate = len(faulty_detected_pkts)/(pkts_tx_rb[0]-pkts_tx[0])*100
print('Detection rate for hard failure:', detection_rate)

Detection rate for hard failure: 75.60975609756098


In [ ]:
######## old
### efficiency of RB
# how many faulty pkts detected?    how many lost packets detected

hb_pkts = []
data_pkts = []

for d in data[redundant_board_ind]:
    #print(d[0][0])
    if d[5][0] == -1:
        hb_pkts += [d]
    else:
        data_pkts += [d]
        
faulty_detected_pkts = []
lost_detected_pkts = []
        
for d in data_pkts:
    if d[1] < 20 or d[1] > 47:   ### 20: last pkt by PB before failure, 47: last pkt by RB bfore resetting pkt count
        lost_detected_pkts += [d]
    else:
        faulty_detected_pkts += [d]

detection_rate = len(faulty_detected_pkts)/(47-20)*100
print('Detection rate for hard failure:', detection_rate)